<a href="https://colab.research.google.com/github/AnikaZN/DS-Unit-1-Sprint-4-Linear-Algebra/blob/master/module4-clustering/Cell_Clustering_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# K-Means Clustering

Your assignment is to use the "Breast Cancer Wisconsin (Diagnostic) Data Set" from Kaggle to try and cluster types of cancer cells. 

It may be helpful to use PCA to reduce the dimensions of your data first in order to obtain --but then again, maybe not. I dunno, you're the data scientist, you tell me.🤪 

Here's the original dataset for your reference:

<https://www.kaggle.com/uciml/breast-cancer-wisconsin-data>

## This is a supervised learning dataset

(Because it has **labels** - The "diagnosis" column.)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA # You don't necessarily have to use this
from sklearn.cluster import KMeans # You don't necessarily have to use this
from sklearn.preprocessing import StandardScaler # You don't necessarily have to use this

df = pd.read_csv("https://raw.githubusercontent.com/ryanleeallred/datasets/master/Cancer_Cells.csv")
print(df.shape)
df.head()

(569, 33)


,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


## Now it's an unsupervised learning dataset

(Because we've removed the diagnosis label) - Use this version.

In [2]:
df = df.drop('diagnosis', axis=1)
df.head()

,id,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


# You take it from here!

See what you can come up with. You have all the know-how! 

- You might want to do some data exploration to see if you can find specific columns that will help you find distinct clusters of cells
- You might want to do PCA on this data to see if that helps you find distinct linearly-separable clusters.
  - (In the real world, truly linearly-separable clusters are rare.)
- You might want to use an elbow chart to decide on the number of clusters to use.
- You might want to use a scree plot to decide how many principal components to include in your clustering.
- You might want to standardize your data before PCA (If you decide to use PCA). 

## Manage your time and don't spend it all on data exploration or something like that. You got this!

In [3]:
df.shape

(569, 32)

In [4]:
df.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568
id,842302.000000,842517.000000,8.430090e+07,8.434830e+07,8.435840e+07,843786.000000,844359.000000,8.445820e+07,844981.000000,8.450100e+07,845636.000000,8.461000e+07,846226.000000,846381.000000,8.466740e+07,8.479900e+07,848406.000000,8.486200e+07,849014.000000,8.510426e+06,8.510653e+06,8.510824e+06,8.511133e+06,851509.000000,852552.000000,852631.000000,852763.000000,852781.000000,852973.000000,853201.000000,853401.000000,853612.000000,8.538260e+07,854002.000000,854039.000000,854253.000000,854268.000000,854941.000000,855133.000000,855138.000000,...,918465.000000,91858.000000,9.190390e+07,9.190390e+07,9.193040e+07,919537.000000,919555.000000,9.197970e+07,919812.000000,921092.000000,921362.000000,921385.000000,921386.000000,921644.000000,922296.000000,922297.000000,922576.000000,922577.000000,922840.00000,923169.000000,923465.000000,923748.000000,923780.000000,924084.000000,924342.000000,924632.000000,924934.000000,924964.000000,925236.000000,925277.000000,925291.000000,925292.000000,925311.000000,925622.000000,926125.000000,926424.000000,926682.000000,926954.000000,927241.000000,92751.000000
radius_mean,17.990000,20.570000,1.969000e+01,1.142000e+01,2.029000e+01,12.450000,18.250000,1.371000e+01,13.000000,1.246000e+01,16.020000,1.578000e+01,19.170000,15.850000,1.373000e+01,1.454000e+01,14.680000,1.613000e+01,19.810000,1.354000e+01,1.308000e+01,9.504000e+00,1.534000e+01,21.160000,16.650000,17.140000,14.580000,18.610000,15.300000,17.570000,18.630000,11.840000,1.702000e+01,19.270000,16.130000,16.740000,14.250000,13.030000,14.990000,13.480000,...,12.070000,11.750000,1.167000e+01,1.368000e+01,2.047000e+01,10.960000,20.550000,1.427000e+01,11.690000,7.729000,7.691000,11.540000,14.470000,14.740000,13.210000,13.870000,13.620000,10.320000,10.26000,9.683000,10.820000,10.860000,11.130000,12.770000,9.333000,12.880000,10.290000,10.160000,9.423000,14.590000,11.510000,14.050000,11.200000,15.220000,20.920000,21.560000,20.130000,16.600000,20.600000,7.760000
texture_mean,10.380000,17.770000,2.125000e+01,2.038000e+01,1.434000e+01,15.700000,19.980000,2.083000e+01,21.820000,2.404000e+01,23.240000,1.789000e+01,24.800000,23.950000,2.261000e+01,2.754000e+01,20.130000,2.068000e+01,22.150000,1.436000e+01,1.571000e+01,1.244000e+01,1.426000e+01,23.040000,21.380000,16.400000,21.530000,20.250000,25.270000,15.050000,25.110000,18.700000,2.398000e+01,26.470000,17.880000,21.590000,21.720000,18.420000,25.200000,20.820000,...,13.440000,17.560000,2.002000e+01,1.633000e+01,2.067000e+01,17.620000,20.860000,2.255000e+01,24.440000,25.490000,25.440000,14.440000,24.990000,25.420000,28.060000,20.700000,23.230000,16.350000,16.58000,19.340000,24.210000,21.480000,22.440000,29.430000,21.940000,28.920000,27.610000,19.590000,27.880000,22.680000,23.930000,27.150000,29.370000,30.620000,25.090000,22.390000,28.250000,28.080000,29.330000,24.540000
perimeter_mean,122.800000,132.900000,1.300000e+02,7.758000e+01,1.351000e+02,82.570000,119.600000,9.020000e+01,87.500000,8.397000e+01,102.700000,1.036000e+02,132.400000,103.700000,9.360000e+01,9.673000e+01,94.740000,1.081000e+02,130.000000,8.746000e+01,8.563000e+01,6.034000e+01,1.025000e+02,137.200000,110.000000,116.000000,97.410000,122.100000,102.400000,115.000000,124.800000,77.930000,1.128000e+02,127.900000,107.000000,110.100000,93.630000,82.610000,95.540000,88.400000,...,77.830000,75.890000,7.521000e+01,8.776000e+01,1.347000e+02,70.790000,137.800000,9.377000e+01,76.370000,47.980000,48.340000,74.650000,95.810000,94.700000,84.880000,89.770000,87.190000,65.310000,65.85000,61.050000,68.890000,68.510000,71.490000,81.350000,59.010000,82.500000,65.670000,64.730000,59.260000,96.390000,74.520000,91.380000,70.670000,103.400000,143.000000,142.000000,131.200000,108.300000,140.100000,

In [5]:
df.dtypes

id                           int64
radius_mean                float64
texture_mean               float64
perimeter_mean             float64
area_mean                  float64
smoothness_mean            float64
compactness_mean           float64
concavity_mean             float64
concave points_mean        float64
symmetry_mean              float64
fractal_dimension_mean     float64
radius_se                  float64
texture_se                 float64
perimeter_se               float64
area_se                    float64
smoothness_se              float64
compactness_se             float64
concavity_se               float64
concave points_se          float64
symmetry_se                float64
fractal_dimension_se       float64
radius_worst               float64
texture_worst              float64
perimeter_worst            float64
area_worst                 float64
smoothness_worst           float64
compactness_worst          float64
concavity_worst            float64
concave points_worst

In [6]:
df.isna().sum()

id                           0
radius_mean                  0
texture_mean                 0
perimeter_mean               0
area_mean                    0
smoothness_mean              0
compactness_mean             0
concavity_mean               0
concave points_mean          0
symmetry_mean                0
fractal_dimension_mean       0
radius_se                    0
texture_se                   0
perimeter_se                 0
area_se                      0
smoothness_se                0
compactness_se               0
concavity_se                 0
concave points_se            0
symmetry_se                  0
fractal_dimension_se         0
radius_worst                 0
texture_worst                0
perimeter_worst              0
area_worst                   0
smoothness_worst             0
compactness_worst            0
concavity_worst              0
concave points_worst         0
symmetry_worst               0
fractal_dimension_worst      0
Unnamed: 32                569
dtype: i

In [7]:
df = df.drop('Unnamed: 32', axis = 1)

df.isna().sum()

id                         0
radius_mean                0
texture_mean               0
perimeter_mean             0
area_mean                  0
smoothness_mean            0
compactness_mean           0
concavity_mean             0
concave points_mean        0
symmetry_mean              0
fractal_dimension_mean     0
radius_se                  0
texture_se                 0
perimeter_se               0
area_se                    0
smoothness_se              0
compactness_se             0
concavity_se               0
concave points_se          0
symmetry_se                0
fractal_dimension_se       0
radius_worst               0
texture_worst              0
perimeter_worst            0
area_worst                 0
smoothness_worst           0
compactness_worst          0
concavity_worst            0
concave points_worst       0
symmetry_worst             0
fractal_dimension_worst    0
dtype: int64

In [0]:
import seaborn as sns

#sns.pairplot(df)

From the pairplot, I made note of the following features:

radius_mean - perimeter_mean - area_mean - compactness_mean - concavity_mean - concavity_se - concave_points_se - radius_worst - perimeter_worst - area_worst - compactness_worst - concavity_worst - concave_points_worst

In [9]:
fun_data = df[['radius_mean'],
             ['perimeter_mean'],
             ['area_mean'],
             ['compactness_mean'],
             ['concavity_mean'],
             ['concavity_se'],
             ['concave_points_se'],
             ['radius_worst'],
             ['perimeter_worst'],
             ['area_worst'],
             ['compactness_worst'],
             ['concavity_worst'],
             ['concave_points_worst']]

TypeError: ignored

In [10]:
from numpy import array
from sklearn.decomposition import PCA
import sklearn.preprocessing

sklearn.preprocessing.scale(df)

data = df
pca = PCA(2)
pca.fit(data)
T = pca.transform(data)

print('\n Eigenvectors: \n', pca.components_)
print('\n Eigenvalues: \n', pca.explained_variance_)
print('\n Projected Data: \n', T)


 Eigenvectors: 
 [[ 1.00000000e+00  2.10355216e-09  3.43234571e-09  1.42192516e-08
   2.72738712e-07 -1.45885088e-12  4.04272551e-14  3.19336553e-11
   1.37054208e-11 -4.84913013e-12 -2.96551059e-12  3.17299072e-10
  -3.32090176e-11  2.22094177e-09  6.46746166e-08  2.32430050e-12
   4.86463229e-12  1.33374625e-11  3.88753147e-12 -1.14429366e-12
   5.44478603e-13  3.18575458e-09  3.18174001e-09  2.14982890e-08
   4.88138784e-07  1.88802857e-12 -3.73530805e-12  3.87188697e-11
   1.84932882e-11 -2.18847361e-11 -4.31457934e-12]
 [-5.62073572e-07  5.10143085e-03  2.15255874e-03  3.51868793e-02
   5.17187910e-01  4.31365211e-06  4.09811144e-05  8.22229476e-05
   4.80443503e-05  7.25444830e-06 -2.59184761e-06  3.10913866e-04
  -6.51708674e-05  2.21700252e-03  5.50584735e-02 -8.61280151e-07
   5.48385367e-06  8.70373492e-06  3.23823727e-06 -1.23211669e-06
  -9.73388486e-08  7.17147932e-03  3.03812662e-03  4.95838985e-02
   8.51875672e-01  6.45402963e-06  1.02489500e-04  1.70054198e-04
   7.41

# Stretch Goal:

Once you are satisfied with your clustering, go back and add back in the labels from the original dataset to check how accurate your clustering was. Remember that this will not be a possibility in true unsupervised learning, but it might be a helpful for your learning to be able to check your work against the "ground truth". Try different approaches and see which one is the most successful and try understand why that might be the case. If you go back and try different methods don't ever include the actual "diagnosis" labels in your clustering or PCA.

**Side Note** Data Science is never DONE. You just reach a point where the cost isn't worth the benefit anymore. There's always more moderate to small improvements that we could make. Don't be a perfectionist, be a pragmatist.